# Мини-таск №1: Data preprocessing

## Описание

1.  Очистить и преобразовать числовые параметры, в том числе таргетную величину Kd, из строкового типа данных в числовой тип. Это может включать удаление лишних символов, конвертацию в правильный числовой формат и приведение значений к единому виду. Рекомендуется использовать для этого регулярные выражения и библиотеку re.
2.  Обработать другие параметры, которые покажутся важными для предсказания целевой переменной (например, провести энкодинг состава буфера или типа аптамера)
3.  Для малых молекул преобразовать названия в SMILES-представление, а для белковых мишеней - в последовательность аминокислот. Дополнить датафрейм этой информацией. Для достоверности полученных данных советуем пользоваться комбинацией библиотек requests и API баз данных (например, PubChem для поиска SMILES по названию и UniProt для поиска последовательности аминокислот белка по названию)
4.  Найти дополнительную информацию о мишенях через различные открытые базы данных и источники. Дополнить датафрейм этой информацией, чтобы обогатить данные для последующего анализа.
5.  Проверить качество и полноту полученного датафрейма, устранить возможные пропуски данных и неоднозначности.


## 0
Загрузим необходимые библиотеки

In [3]:
# Общие библиотеки
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import re

## 1
Загрузим наш датасет и посмотрим, как он выглядит:

In [20]:
df = pd.read_excel('UTexas Aptamer Database.xlsx', header = 0)

In [21]:
df.head()

,Year of Paper,Link to PubMed Entry,Journals,Journal DOI,Citation,Type of Nucleic Acid,Name of Aptamer,Target,Aptamer Sequence,Sequence Length,...,pH,Molecular weight of target,Application as quoted in the referenced paper,Post-selex modifications to the aptamer,Additional Information\n\n,Serial Number,Parent sequence serial number,"Corresponding Author Name, email address",please fill out the form for any feedbacks/comments,"Aptagen Cross Referencing(Check Aptamer Chemistry, Affinity, Length, GC content, sequence)"
0,1990,https://pubmed.ncbi.nlm.nih.gov/1697402/,Nature,https://doi.org/10.1038/346818a0,"Ellington, A. D., & Szostak, J. W. (1990). In ...",ssRNA,CB-42,Cibacron Blue 3GA,5'GGGAGAAUUCCCGCGGCAGAAGCCCACCUGGCUUUGAACUCUAU...,132,...,7.6,Not reported,"Detection: "" Isolate RNAs that bind to several...",Not applicable,The aptamer was reported in DNA (include thymi...,10000000,NaN,Szostak JW,https://forms.gle/n4TzuyddXQrHYJXF9,NaN
1,1990,https://pubmed.ncbi.nlm.nih.gov/1697402/,Nature,https://doi.org/10.1038/346818a0,"Ellington, A. D., & Szostak, J. W. (1990). In ...",ssRNA,B4-25,Reactive Blue 4,5'GGGAGAAUUCCCGCGGCGUUGGCCCAGGAUAAUAGGACGAAAUC...,133,...,7.6,Not reported,"Detection: "" Isolate RNAs that bind to several...",Not applicable,DNA library/pool was used as a template to gen...,10000001,NaN,Szostak JW,https://forms.gle/n4TzuyddXQrHYJXF9,NaN
2,1990,https://pubmed.ncbi.nlm.nih.gov/2200121/,Science,https://doi.org/10.1126/science.2200121,"Tuerk, C., & Gold, L. (1990). Systematic evolu...",ssRNA,wild type,T4 DNA polymerase (gp43),5'GAAUUGUGGUGUUGGCUCCCUAUAGUGAGUCGUAUUAAUAUUCC...,113,...,NaN,Not reported,"Detection: "" We have previously shown that the...",Not applicable,DNA library/pool was used as a template to gen...,10000002,NaN,Gold L,https://forms.gle/n4TzuyddXQrHYJXF9,NaN
3,1990,https://pubmed.ncbi.nlm.nih.gov/2200121/,Science,https://doi.org/10.1126/science.2200121,"Tuerk, C., & Gold, L. (1990). Systematic evolu...",ssRNA,major variant,T4 DNA polymerase (gp43),5'GAAUUGUGGUGUUGGCUCCCUAUAGUGAGUCGUAUUAAUAUUCC...,113,...,NaN,Not reported,"Detection: "" We have previously shown that the...",Not applicable,DNA library/pool was used as a template to gen...,10000003,NaN,Gold L,https://forms.gle/n4TzuyddXQrHYJXF9,NaN
4,1992,https://pubmed.ncbi.nlm.nih.gov/1741036/,Nature,https://doi.org/10.1038/355564a0,"Bock, L. C., Griffin, L. C., Latham, J. A., Ve...",ssDNA,15 mer (colloquially known as Bock DNA Aptamer),"Thrombin (Sigma), Human",5'GGTTGGTGTGGTTGG3',15,...,7.4,Not reported,"Diagnostic and therapeutic: ""We are at present...",Not applicable,Presumed minimized variant was found,10000004,NaN,Toole JJ,https://forms.gle/n4TzuyddXQrHYJXF9,NaN


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1495 entries, 0 to 1494
Data columns (total 28 columns):
 #   Column                                                                                       Non-Null Count  Dtype  
---  ------                                                                                       --------------  -----  
 0   Year of Paper                                                                                1495 non-null   int64  
 1   Link to PubMed Entry                                                                         1495 non-null   object 
 2   Journals                                                                                     1495 non-null   object 
 3   Journal DOI                                                                                  1495 non-null   object 
 4   Citation                                                                                     1495 non-null   object 
 5   Type of Nucleic Acid              

#### Численные признаки, которые не нужно преобразовывать:
- Year of Paper
- Sequence Length
- GC Content
- Serial Number
- pH

#### Численные признаки, которые нужно преобразовывать:
- Affinity
- Kd (nM)
- Pool Random Region
- Molecular weight of target

**Kd (nM)** и **Pool Random Region** содержат по одной стоке с тектом вместо чисел => удалим их

In [ ]:
# Удалим строки 1489 и  1489  
df = df.drop(df[df['Link to PubMed Entry'] =='https://patents.google.com/patent/US9636419B2/en'].index)
df = df.drop(df[df['Link to PubMed Entry'] =='https://patents.google.com/patent/US20090105172A1/en'].index)

In [35]:
# меняем формат Kd (nM)
df['Kd (nM)'] = df['Kd (nM)'].astype(float)

In [36]:
# меняем формат Pool Random Region
df['Pool Random Region '].astype(float)

**Molecular weight of target** содержит 1247 незаполненных строк из 1485 => удалим весь столбец

In [56]:
df['Molecular weight of target'].value_counts() # Не нужно преобразовывать

Molecular weight of target
Not reported     1210
Not reported       37
26-28 kDa          17
25 kDa             16
94 kDa             11
                 ... 
20051.8 G/mol       1
19967.6 G/mol       1
82 kDa              1
18.7 Kda            1
64 kDa              1
Name: count, Length: 89, dtype: int64

In [ ]:
df = df.drop(['Molecular weight of target'], axis = 1)

**Affinity** 

In [58]:
df['Affinity'].value_counts()

Affinity
Not reported                                                                                                                                                    265
Kd: 19-36 nM                                                                                                                                                     13
Not reported                                                                                                                                                     13
Kd: 228.4 nM and 7485.3 nM *this Kd was determined with affinity test on entire n round pool                                                                     11
Kd: 10 nM                                                                                                                                                         9
                                                                                                                                                               ... 
Kd: 2.1

##### Мы видим, что форматы данных сильно варьируются. Используем функцию с регулярными выражениями, чтобы преобразовать данные в один вид

In [80]:
def extract_numerical_value(text):
    pattern = r'(\d+\.\d+|\d+)'  # Шаблон регулярного выражения для сопоставления чисел с плавающей точкой или целых чисел

    # Check if the text contains a range of values
    range_match = re.match(r'Kd: (\d+\.\d+|\d+)-(\d+\.\d+|\d+) nM', text)
    if range_match:  # Если есть диапазон, посчитаем среднее значение
        start_value = float(range_match.group(1))
        end_value = float(range_match.group(2))
        return (start_value + end_value) / 2
    else:  # Если это одно значение, используем регулярное выражение
        match = re.search(pattern, text)
        if match:
            return float(match.group(0))
    return None  # Вернуть None если нет чисел

# Применим функцию к столбцу
df['Affinity'] = df['Affinity'].apply(extract_numerical_value)

## 2

Рассмотрим категориальные признаки, которые кажутся важными для предсказания целевой переменной

**Type of the buffer** содержит несколько видов строковых значений, для работы с такой задачей, можно использовать метод `map`

In [83]:
df['Type of the buffer'].value_counts()

Type of the buffer
Tris Buffers             637
PBS/phosphate buffers    496
Other Buffers            298
Not Reported              59
Name: count, dtype: int64

In [ ]:
d = {'Tris Buffers' : 1, 'PBS/phosphate buffers' : 2, 'Other Buffers' : 3, 'Not Reported' : 0}
df['Type of the buffer'] = df['Type of the buffer'].map(d)

Аналогично с **Divalent Salt**

In [84]:
df['Divalent Salt '].value_counts()

Divalent Salt 
MgCl         669
MgCl/CaCl    232
CaCl          48
MgCl           5
Name: count, dtype: int64

In [ ]:
d = {'MgCl' : 1, 'MgCl/CaCl' : 2, 'CaCl' : 3, 'MgCl' : 4}
df['Type of the buffer'] = df['Type of the buffer'].map(d)

Аналогично с **Type of Nucleic Acid**

In [89]:
df['Type of Nucleic Acid'].value_counts()

Type of Nucleic Acid
ssDNA                    896
ssRNA                    438
2'-fluoro-RNA            107
2'-amino-RNA              23
5-uracil-modified-DNA      7
2'-O-Me-RNA                6
2',4'-BNA/LNA-DNA          5
2'-fluoro/O-Me-RNA         3
FANA XNA                   3
4'-thio-RNA                2
2'-fluoro/amino-RNA        1
5-uracil-modified-RNA      1
dsDNA                      1
Name: count, dtype: int64

In [ ]:
d = {'ssDNA' : 1, 'ssRNA' : 2, '2\'-fluoro-RNA' : 3, '2\'-amino-RNA' : 4, '5-uracil-modified-DNA' : 5,'2\'-O-Me-RNA' : 6, '2\',4\'-BNA/LNA-DNA' : 7, 
    '2\'-fluoro/O-Me-RNA' : 8, 'FANA XNA' : 9, '4\'-thio-RNA' : 11, '2\'-fluoro/amino-RNA' : 12, '5-uracil-modified-RNA' : 13, 'dsDNA' : 14}
df['Type of the buffer'] = df['Type of the buffer'].map(d)

In [94]:
unique_aptamer = df['Name of Aptamer'].unique()
len(unique_aptamer)

1348

In [95]:
df['Name of Aptamer'].value_counts()

Name of Aptamer
C2                          6
Apt1                        5
C1                          5
Aptamer 1                   4
#3                          4
                           ..
4.4.6                       1
4.4.5                       1
4.4.4                       1
4.4.3                       1
GS24 (truncated version)    1
Name: count, Length: 1348, dtype: int64

**Name of Aptamer** содержит важную информацию, необходимую для расчета `Kd`
Преобразуем данные в дамми-переменную

In [ ]:
# Создаем переменные
dummy_variables = pd.get_dummies(df['Name of Aptamer'])
df = pd.concat([df, dummy_variables], axis=1)